# Cargill-SMU Datathon 2026: Voyage Optimization Model

**Team:** [Your Team Name]  
**Members:** [List of Members]  
**Date:** January 2026

---

## Table of Contents

1. [Introduction & Problem Statement](#1-introduction)
2. [Setup & Data Loading](#2-setup)
3. [Core Freight Calculator](#3-calculator)
4. [Voyage Optimization & Recommendation](#4-optimization)
5. [Scenario Analysis](#5-scenarios)
6. [Chatbot Integration](#6-chatbot)
7. [Conclusion & Key Findings](#7-conclusion)
8. [Appendix: Assumptions](#8-assumptions)

---

<a id='1-introduction'></a>
## 1. Introduction & Problem Statement

### Challenge Overview

As Dry Bulk Traders at Cargill Ocean Transportation Singapore, we are tasked with optimizing the employment of **4 Capesize vessels** and allocating **3 committed cargoes** to maximize portfolio profit.

### Key Deliverables

1. **Freight Calculator**: A model that evaluates voyage profitability based on vessel positions and cargo opportunities
2. **Voyage Recommendation**: Optimal vessel-cargo allocation that maximizes total portfolio profit
3. **Scenario Analysis**: Sensitivity analysis on bunker prices and port delays
4. **Chatbot**: An interactive AI assistant that responds to voyage queries

### Our Approach

We use a systematic approach:
- Calculate Time Charter Equivalent (TCE) for all feasible vessel-cargo combinations
- Identify the optimal allocation using a greedy optimization strategy
- Perform sensitivity analysis to find threshold values for key parameters

---

<a id='2-setup'></a>
## 2. Setup & Data Loading

In [ ]:
# Standard library imports
import sys
import warnings
from pathlib import Path

# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
warnings.filterwarnings('ignore')

# Add src directory to path
sys.path.append(str(Path.cwd().parent / 'src'))

print("Libraries imported successfully!")

In [ ]:
# Import our custom modules
from data_loader import load_all_data, get_distance
from freight_calculator import FreightCalculator, VoyageResult

# Load all data
data = load_all_data()

In [ ]:
# Display data summary
print("=" * 60)
print("DATA SUMMARY")
print("=" * 60)
print(f"\nPort distance pairs: {len(data['distances']):,}")
print(f"\nVessels: {len(data['vessels'])}")
print(f"  - Cargill vessels: {len(data['vessels'][data['vessels']['vessel_type'] == 'cargill'])}")
print(f"  - Market vessels: {len(data['vessels'][data['vessels']['vessel_type'] == 'market'])}")
print(f"\nCargoes: {len(data['cargoes'])}")
print(f"  - Cargill committed: {len(data['cargoes'][data['cargoes']['cargo_type'] == 'cargill'])}")
print(f"  - Market cargoes: {len(data['cargoes'][data['cargoes']['cargo_type'] == 'market'])}")

### 2.1 Cargill Vessels Overview

In [ ]:
# Display Cargill vessels
cargill_vessels = data['vessels'][data['vessels']['vessel_type'] == 'cargill']
display_cols = ['vessel_name', 'dwt', 'hire_rate', 'current_port', 'etd', 
                'speed_laden_eco', 'speed_ballast_eco']
cargill_vessels[display_cols]

### 2.2 Cargill Committed Cargoes Overview

In [ ]:
# Display Cargill committed cargoes
cargill_cargoes = data['cargoes'][data['cargoes']['cargo_type'] == 'cargill']
display_cols = ['cargo_id', 'route', 'customer', 'commodity', 'quantity', 
                'freight_rate', 'laycan_start', 'laycan_end']
cargill_cargoes[display_cols]

### 2.3 Bunker Prices (March 2026)

In [ ]:
# Display bunker prices
data['bunker_prices']

---

<a id='3-calculator'></a>
## 3. Core Freight Calculator

Our freight calculator computes voyage profitability using the following key metrics:

- **Time Charter Equivalent (TCE)**: The industry-standard metric for comparing voyage profitability
- **Voyage Profit**: Gross freight minus all voyage costs

### Key Equations

```
TCE = (Gross Freight - Voyage Costs) / Total Voyage Days

Voyage Costs = Bunker Cost + Port Costs + Commission

Gross Freight = Cargo Quantity × Freight Rate
```

In [ ]:
# Initialize the freight calculator
calculator = FreightCalculator(
    data=data,
    use_eco_speed=True,  # Use economical speed for fuel efficiency
    weather_margin=0.05  # 5% weather buffer
)

print("Freight Calculator initialized with:")
print("  - Speed mode: Economical")
print("  - Weather margin: 5%")

### 3.1 Example Calculation: Single Voyage

In [ ]:
# Example: Calculate voyage for ANN BELL carrying the West Africa-China Bauxite cargo
vessel = cargill_vessels[cargill_vessels['vessel_name'] == 'ANN BELL'].iloc[0]
cargo = cargill_cargoes[cargill_cargoes['cargo_id'] == 'CARGILL_1'].iloc[0]

result = calculator.calculate_voyage(vessel, cargo)

print(f"\n{'='*60}")
print(f"VOYAGE CALCULATION: {result.vessel_name} → {result.cargo_id}")
print(f"{'='*60}")
print(f"\n📍 Route: {vessel['current_port']} → {cargo['load_port']} → {cargo['discharge_port']}")
print(f"\n⏱️  TIME BREAKDOWN:")
print(f"   Ballast leg:     {result.ballast_days:.1f} days")
print(f"   Laden leg:       {result.laden_days:.1f} days")
print(f"   Load port:       {result.load_port_days:.1f} days")
print(f"   Discharge port:  {result.discharge_port_days:.1f} days")
print(f"   TOTAL:           {result.total_voyage_days:.1f} days")
print(f"\n⛽ BUNKER CONSUMPTION:")
print(f"   VLSFO: {result.vlsf_consumed:,.0f} MT")
print(f"   MGO:   {result.mgo_consumed:,.0f} MT")
print(f"\n💰 FINANCIALS:")
print(f"   Gross Freight:   ${result.gross_freight:>12,.0f}")
print(f"   Bunker Cost:     ${result.bunker_cost:>12,.0f}")
print(f"   Port Costs:      ${result.port_costs:>12,.0f}")
print(f"   Commission:      ${result.commission:>12,.0f}")
print(f"   ─────────────────────────────")
print(f"   Voyage Profit:   ${result.voyage_profit:>12,.0f}")
print(f"   TCE:             ${result.tce:>12,.0f}/day")
print(f"\n✅ Feasible: {result.is_feasible}")
print(f"   {result.feasibility_notes}")

---

<a id='4-optimization'></a>
## 4. Voyage Optimization & Recommendation

We now calculate the profitability of all feasible vessel-cargo combinations and identify the optimal allocation.

In [ ]:
# Calculate all combinations for Cargill vessels and Cargill cargoes
results_df = calculator.calculate_all_combinations(
    vessels_df=cargill_vessels,
    cargoes_df=cargill_cargoes
)

# Filter to feasible voyages only
feasible_results = results_df[results_df['is_feasible'] == True].copy()

print(f"Total combinations evaluated: {len(results_df)}")
print(f"Feasible combinations: {len(feasible_results)}")

In [ ]:
# Display results sorted by TCE
display_cols = ['vessel_name', 'cargo_id', 'route', 'total_voyage_days', 
                'gross_freight', 'voyage_profit', 'tce', 'is_feasible']

feasible_results[display_cols].sort_values('tce', ascending=False)

### 4.1 Optimal Vessel-Cargo Allocation

We use a greedy algorithm to find the best allocation:
1. Sort all combinations by TCE (descending)
2. Assign each cargo to the best available vessel
3. Ensure each vessel is assigned at most one cargo

In [ ]:
# TODO: Implement optimal allocation algorithm
# This is a placeholder for the team to complete

def find_optimal_allocation(results_df):
    """
    Find the optimal vessel-cargo allocation to maximize total portfolio profit.
    
    Args:
        results_df: DataFrame with all voyage calculations
        
    Returns:
        DataFrame with optimal allocation
    """
    # Filter to feasible only
    feasible = results_df[results_df['is_feasible'] == True].copy()
    
    # Sort by TCE descending
    feasible = feasible.sort_values('tce', ascending=False)
    
    # Greedy allocation
    allocated_vessels = set()
    allocated_cargoes = set()
    allocation = []
    
    for _, row in feasible.iterrows():
        if row['vessel_name'] not in allocated_vessels and row['cargo_id'] not in allocated_cargoes:
            allocation.append(row)
            allocated_vessels.add(row['vessel_name'])
            allocated_cargoes.add(row['cargo_id'])
    
    return pd.DataFrame(allocation)

# Find optimal allocation
optimal_allocation = find_optimal_allocation(results_df)
optimal_allocation[display_cols]

In [ ]:
# Calculate total portfolio metrics
total_profit = optimal_allocation['voyage_profit'].sum()
avg_tce = optimal_allocation['tce'].mean()

print(f"\n{'='*60}")
print("OPTIMAL PORTFOLIO SUMMARY")
print(f"{'='*60}")
print(f"\nTotal Voyage Profit: ${total_profit:,.0f}")
print(f"Average TCE: ${avg_tce:,.0f}/day")
print(f"\nAllocations:")
for _, row in optimal_allocation.iterrows():
    print(f"  • {row['vessel_name']} → {row['cargo_id']} ({row['route']})")
    print(f"    TCE: ${row['tce']:,.0f}/day | Profit: ${row['voyage_profit']:,.0f}")

---

<a id='5-scenarios'></a>
## 5. Scenario Analysis

We analyze how changes in key parameters affect our recommendations.

### 5.1 Scenario 1: Bunker Price Sensitivity

In [ ]:
# TODO: Implement bunker price sensitivity analysis
# Test bunker price changes from -20% to +50%

print("Bunker Price Sensitivity Analysis")
print("="*60)
print("\n[TO BE IMPLEMENTED BY TEAM]")
print("\nObjective: Find the bunker price threshold at which")
print("the current recommendation becomes suboptimal.")

### 5.2 Scenario 2: Port Delay Impact

In [ ]:
# TODO: Implement port delay sensitivity analysis
# Test additional port days from 0 to 10 days

print("Port Delay Sensitivity Analysis")
print("="*60)
print("\n[TO BE IMPLEMENTED BY TEAM]")
print("\nObjective: Find the number of additional port delay days")
print("at which the current recommendation becomes suboptimal.")

---

<a id='6-chatbot'></a>
## 6. Chatbot Integration

This section demonstrates how to interact with our model via the chatbot.

In [ ]:
# Example chatbot interaction
def chatbot_response(query: str, data: dict, calculator: FreightCalculator) -> str:
    """
    Generate a response to a user query about voyage recommendations.
    
    This is a simplified version - the full chatbot uses OpenAI API.
    """
    query_lower = query.lower()
    
    # Handle recommendation queries
    if 'recommend' in query_lower or 'best' in query_lower:
        return f"""Based on our analysis, the optimal vessel-cargo allocation is:

1. ANN BELL should carry the West Africa-China Bauxite cargo (CARGILL_1)
   - TCE: $XX,XXX/day
   - Voyage Profit: $X,XXX,XXX

2. [Additional recommendations...]

Total Portfolio Profit: $X,XXX,XXX
"""
    
    # Handle what-if queries
    if 'what if' in query_lower or 'bunker' in query_lower:
        return """If bunker prices increase by 20%, the recommendation changes:
- Current best: ANN BELL → CARGILL_1 (TCE: $XX,XXX/day)
- New best: PACIFIC GLORY → CARGILL_2 (TCE: $XX,XXX/day)

Threshold: Bunker price increase of 15% ($XXX/MT VLSFO)
"""
    
    return "I can help you with voyage recommendations and what-if scenarios. Try asking about the best vessel for a specific cargo!"

# Test the chatbot
print("Chatbot Demo")
print("="*60)
test_query = "What is the best voyage recommendation for our fleet?"
print(f"\nUser: {test_query}")
print(f"\nAssistant: {chatbot_response(test_query, data, calculator)}")

---

<a id='7-conclusion'></a>
## 7. Conclusion & Key Findings

### Summary of Recommendations

[TO BE COMPLETED BY TEAM]

### Key Insights from Scenario Analysis

[TO BE COMPLETED BY TEAM]

### Limitations

- Weather margin is a fixed 5% buffer; actual weather conditions may vary
- Port delays are estimated; real-time congestion data would improve accuracy
- Bunker prices are based on forward curves; spot prices may differ

---

<a id='8-assumptions'></a>
## 8. Appendix: Assumptions

| # | Assumption | Value/Rationale |
|---|------------|----------------|
| 1 | Vessel Speed | Economical speed used for all calculations |
| 2 | Weather Margin | 5% buffer added to all sea voyage durations |
| 3 | Port Waiting | 1 day buffer at each port for waiting/congestion |
| 4 | Bunker Prices | March 2026 forward prices from problem statement |
| 5 | Cargo Quantity | Maximum within tolerance based on vessel DWT |
| 6 | Missing Distances | Singapore used as intermediary for routing |
| 7 | Currency | All calculations in USD |
| 8 | Market Vessel Hire | Baltic 5TC rate ($18,454/day) used as benchmark |